<a href="https://colab.research.google.com/github/onaka-yurusugi/CycleGAN-FantasyCnversion-/blob/master/%E3%80%90WebApp%E3%80%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11683804186222339911, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11345264640
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6582557207192175658
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## CycleGANの準備

In [ ]:
%cd /content/gdrive/My Drive/Colab Notebooks/
!mkdir WebApp
%cd /content/gdrive/My Drive/Colab Notebooks/WebApp
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix.git
%cd /content/gdrive/My Drive/Colab Notebooks/WebApp/pytorch-CycleGAN-and-pix2pix
!pip install -r requirements.txt

!git clone https://github.com/facebookresearch/visdom.git
%cd visdom
!pip install -e .
!pip install visdom dominate

/content/gdrive/My Drive/Colab Notebooks
mkdir: cannot create directory ‘WebApp’: File exists
/content/gdrive/My Drive/Colab Notebooks/WebApp
fatal: destination path 'pytorch-CycleGAN-and-pix2pix' already exists and is not an empty directory.
/content/gdrive/My Drive/Colab Notebooks/WebApp/pytorch-CycleGAN-and-pix2pix
     |████████████████████████████████| 676 kB 8.0 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-py3-none-any.whl size=655250 sha256=02afa3eff1bdc27308d51b7d91622eb68f7095849edf4bed3323f18fdfb198df
  Stored in directory: /root/.cache/pip/wheels/2d/d1/9b/cde923274eac9cbb6ff0d8c7c72fe30a3da9095a38fd50bbf1
  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5710 sha256=14ea77ba5e403fb17c353cad7d281a21d5e24cdb3be181a898c7f127548b5500
  Stored in directory: /root/.cache/pip/wheels/ac/5c/3a/a80e1c65880945c71fd833408cd1e9a8cb7e2f8f37620bb75b
Successfully built visdom torchfile
fatal: de

## ngrokの準備

In [ ]:
%cd /content/gdrive/My Drive/Colab Notebooks/WebApp/
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

/content/gdrive/My Drive/Colab Notebooks/WebApp
--2021-08-27 03:45:44--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.0.235.238, 3.222.79.56, 34.224.52.96, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.0.235.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.3’

ngrok-stable-linux- 100%[===================>]  13.19M  29.3MB/s    in 0.4s    

2021-08-27 03:45:45 (29.3 MB/s) - ‘ngrok-stable-linux-amd64.zip.3’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


## HTML部分

In [ ]:
#Writing to html file on colab
import os

html = """

<!DOCTYPE html>
<html lang="ja">
<head>
	<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
  <link rel="stylesheet" type="text/css" href="{{ url_for('static', filename='css/bootstrap.min.css') }}">
  <link rel="stylesheet" type="text/css" href="{{ url_for('static', filename='css/gantarou.css') }}">
  <link rel="stylesheet" type="text/css" href="{{ url_for('static', filename='css/spectrum.css') }}">
  <link rel="stylesheet" type="text/css" href="{{ url_for('static', filename='css/component.css') }}">
  <link rel="stylesheet" type="text/css" href="{{ url_for('static', filename='css/style.css') }}">
  <script type=text/javascript src="{{ url_for('static', filename='js/jquery-2.1.4.min.js') }}"></script>
  <script type=text/javascript src="{{ url_for('static', filename='js/script.js') }}"></script>
  <script src="https://cdn.rawgit.com/blueimp/JavaScript-Load-Image/v2.6.2/js/load-image.all.min.js"></script>
	
  <script>(function(e,t,n){var r=e.querySelectorAll("html")[0];r.className=r.className.replace(/(^|\s)no-js(\s|$)/,"$1js$2")})(document,window,0);</script>
	<title>
		CycleGAN太郎(Web版)
	</title>
</head>
<body data-luma="174.91" data-colors="31" data-model="0" data-tolerance="5" data-blur="0" data-hash="5dd4d628b1c50b0006fdfabd" data-edges="none" data-layersort="4" data-mins="1" class="advanced">
    <nav class="navbar navbar-default">
      <div class="container">
        <div class="navbar-header">
          <a class="navbar-brand1" href="/">
            <img src="../tmp/img/taroicon.png" width="150" height="80">
          	<span class="cycle">Cycle</span><span class="gantaro">GAN太郎</span><span class="web">(Web版)</span>
          </a>
        </div>
      </div>
    </nav>
    <div class="modeselect">
    <form action="/post" method="post" enctype="multipart/form-data" id="sendForm">
        <ul>
            <li><a href="#tab1" class="current">季節</a></li>
            <li><a href="#tab2">風景</a></li>
            <li><a href="#tab3">異世界</a></li>
            <li><a href="#tab4">山</a></li>
            <li><a href="#tab5">絵画</a></li>
        </ul>
        <div id="contents" class="selection-group">
            
            <div id="tab1">
                <input id="spring" type="radio" name="type" value="spring"  required>
                <label for="spring">
                    <img src="../tmp/img/springicon.png">
                </label>
                <input id="summer" type="radio" name="type" value="summer">
                <label for="summer">
                    <img src="../tmp/img/summericon.png">
                </label>
                <input id="autumn" type="radio" name="type" value="autumn">
                <label for="autumn">
                    <img src="../tmp/img/autumnicon.png">
                </label>
                <input id="winter" type="radio" name="type" value="winter">
                <label for="winter">
                    <img src="../tmp/img/wintericon.png">
                </label>
            </div>
            
            <div id="tab2">
                <input id="aurora" type="radio" name="type" value="aurora"  required>
                <label for="aurora">
                    <img src="../tmp/img/auroraicon.png">
                </label>
                <input id="kagoshima" type="radio" name="type" value="kagoshima">
                <label for="kagoshima">
                    <img src="../tmp/img/kagoshimaicon.png">
                </label>
                <input id="europe" type="radio" name="type" value="europe">
                <label for="europe">
                    <img src="../tmp/img/europeicon.png">
                </label>
                <input id="hanabi" type="radio" name="type" value="hanabi">
                <label for="hanabi">
                    <img src="../tmp/img/hanabiicon.png">
                </label>
            </div>
            
            <div id="tab3">
                <input id="fantasy" type="radio" name="type" value="fantasy"  required>
                <label for="fantasy">
                    <img src="../tmp/img/fantasyicon.png">
                </label>
                <input id="horror" type="radio" name="type" value="horror">
                <label for="horror">
                    <img src="../tmp/img/horroricon.png">
                </label>
                <input id="mirai" type="radio" name="type" value="mirai2">
                <label for="mirai">
                    <img src="../tmp/img/miraiicon.png">
                </label>
                <input id="atlantis" type="radio" name="type" value="atlantis">
                <label for="atlantis">
                    <img src="../tmp/img/atlantisicon.png">
                </label>
            </div>
            
            <div id="tab4">
                <input id="sakurajima" type="radio" name="type" value="sakurajima"  required>
                <label for="sakurajima">
                    <img src="../tmp/img/sakurajimaicon.png">
                </label>
                <input id="fujisan" type="radio" name="type" value="fujisan">
                <label for="fujisan">
                    <img src="../tmp/img/fujisanicon.png">
                </label>
                <input id="hyozan" type="radio" name="type" value="ice">
                <label for="hyozan">
                    <img src="../tmp/img/hyozanicon.png">
                </label>
            </div>
            
            <div id="tab5">
                <input id="ukiyoe" type="radio" name="type" value="ukiyoe"  required>
                <label for="ukiyoe">
                    <img src="../tmp/img/ukiyoeicon.png">
                </label>
                <input id="oil" type="radio" name="type" value="oil">
                <label for="oil">
                    <img src="../tmp/img/oilicon.png">
                </label>
            </div>
            
        </div>
            
        <!-- 画像ファイル選択と変換ボタンです-->
				<div class="box" id="set">

        <input type="file" id="file-7" class="inputfile inputfile-6" style="opacity: 0"  name="uploadFile"/ required>
        <label for="file-7"><span></span> <strong><svg xmlns="http://www.w3.org/2000/svg" width="20" height="17" viewBox="0 0 20 17"><path d="M10 0l-5.2 4.9h3.3v5.1h3.8v-5.1h3.3l-5.2-4.9zm9.3 11.5l-3.2-2.1h-2l3.4 2.6h-3.5c-.1 0-.2.1-.2.1l-.8 2.3h-6l-.8-2.2c-.1-.1-.1-.2-.2-.2h-3.6l3.4-2.6h-2l-3.2 2.1c-.4.3-.7 1-.6 1.5l.6 3.1c.1.5.7.9 1.2.9h16.3c.6 0 1.1-.4 1.3-.9l.6-3.1c.1-.5-.2-1.2-.7-1.5z"/></svg> 画像ファイルを選択&hellip;</strong></label>                                        
        <script type=text/javascript src="{{ url_for('static', filename='js/custom-file-input.js') }}"></script>
        
        <button type="submit" id="sendimg" class="btn-gradient-3d-simple">変換</button>
                    
        <div class="loading-page" id="loading">
            <div class="counter">
                <p>loading</p>
                    <h1>0%
                    </h1>
                    <hr/>
                </div>
            </div>
        </div>
    </form>
</div>
    
   	<form method="post" id="mainform"> 
      <div class="container">
        <div class="row">
          <div class="col-md-6 col-sm-6 col-xs-12 input advanced">
              <h1>変換前</h1>
              <div id="input" class="well text-center">
                  <figure><img id="before" src=""></figure>
                  <img id="inputimg" class="img-responsive" src="{{ url_Image1 }}" onerror="this.style.display='none'">
              </div>
              <br>
          </div>				
          <div class="col-md-6 col-sm-6 col-xs-12 output">					
            <div>
                <h1>変換後</h1>
            </div>
            <div id="output" class="well text-center">
                <figure><img id="after" src=""></figure>
                <img id="outputimg" class="img-responsive" src="{{ url_Image2 }}" onerror="this.style.display='none'">
            </div>
          </div>
        </div>
      </div>
    </form>  
    <br>	
</body>
</html>
"""
#Saving html file
if not os.path.isdir( "templates" ):
    os.makedirs( "templates" )
with open("templates/index.html", mode='w') as f:
    f.write(html)

#Making tmp folder to use temporarily stored
if not os.path.isdir( "tmp" ):
    os.makedirs( "tmp" )

## Webアプリ部分・実行

In [ ]:
%cd /content/gdrive/My Drive/Colab Notebooks/WebApp
get_ipython().system_raw('./ngrok http -subdomain=cyclegantaro 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

from flask import Flask, render_template, send_from_directory, request, redirect, url_for
import numpy as np
import os
import cv2 as cv
import subprocess
import datetime
from PIL import Image, ImageOps

app = Flask(__name__, static_folder="tmp")

global trans_type
trans_type = ""

@app.route("/")
def index():
    return render_template( "index.html" )

@app.route('/post', methods=['POST'])
def post():

    # ラジオボタンの値を取得する
    
    #global trans_type
    trans_type = request.form['type']

    try:
        uploadFile = request.files['uploadFile']
    except:
        return redirect(url_for('index'))

    # 時間ごとに一意なディレクトリを作成

    dirname = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    print(dirname)
    os.mkdir("tmp/up/%s/"%(dirname))
    uploadFile.save("tmp/up/%s/"%(dirname) + uploadFile.filename)

    # Exifタグを取得して画像の回転修正、上書き（スマホからのアップ時に必要）
    image = Image.open("tmp/up/%s/"%(dirname) + uploadFile.filename)

    try:
      orientation = image._getexif()[274]
      if orientation == 6:
          image = image.rotate(-90)
      elif orientation == 8:
          image = image.rotate(-270)
      elif orientation == 3:
          image = image.rotate(180)
      image.save("tmp/up/%s/"%(dirname) + uploadFile.filename, 'PNG')
    except:
      print("Exif処理必要なし")

    # ここまでが画像の受信
    # ここから変換

    %cd /content/gdrive/My Drive/Colab Notebooks/WebApp/pytorch-CycleGAN-and-pix2pix
    # if文で「trans_type」からdirectionAtoB or BtoAを判断
    print(trans_type)
    
    cmd = "python test.py --dataroot ../tmp/up/%s --name %s --model test --no_dropout --results_dir ../tmp/out_dir"%(dirname, trans_type)
    print(cmd)
    print("CycleGANを" + trans_type + "変換で実行します")
    subprocess.call(cmd.split())

    %cd /content/gdrive/My Drive/Colab Notebooks/WebApp

    print(uploadFile.filename)
    filename = uploadFile.filename.split(".")[0] + "_fake.png"
    filename_real = filename.split("_fake.png")[0] + "_real.png"
    print(filename)
    # 変換前画像
    url_Image1 = "/tmp/out_dir/%s/test_latest/images/"%(trans_type) + filename_real
    # 変換後画像
    url_Image2 = "/tmp/out_dir/%s/test_latest/images/"%(trans_type) + filename

    return render_template( "index.html",
                                        url_Image1 = url_Image1, url_Image2 = url_Image2)
    #return render_template( "index.html",
    #                                    url_Image2 = url_Image2)

@app.route('/tmp/out_dir/%s/test_latest/images/<filename>' % (trans_type))
def uploaded_file(filename):
    print(trans_type)
    return send_from_directory("./tmp/out_dir/%s/test_latest/images/"%(trans_type), filename)

if __name__ == '__main__':
    app.run(port=6006)

/content/gdrive/My Drive/Colab Notebooks/WebApp
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:6006/ (Press CTRL+C to quit)
